In [20]:
import torch
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPTNeoForCausalLM, TrainingArguments, Trainer
import numpy as np
import random
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os
from tqdm import tqdm
import re

In [21]:
def split_words(text):
    return re.split(r'[ ](?=[\w])', text)

In [22]:
def split_to_sentences(text):
    """
    Args: 
        text (str): raw text to split to sentences on end of sentences marks.
    Returns:
        List of sentences from text.
    """
    # Multiple options Negative lookbehind to prevent splitting on Mr. or Mrs.
    split_pattern = r'(?<!Mr)(?<!Mrs)[.!?;"]+'
    # Split on end of sentence, but keep the punctuation marks.
    sentences = list(map(str.strip, re.sub(
        split_pattern, r'\g<0>[cut]', text.strip()).split('[cut]')))
    # If the last sentence is ''
    if len(sentences) > 1 and len(sentences[-1]) < 3:
        sentences.pop()
    return sentences

In [23]:
def _readabilty(text, texts_sentences):
    """
    Uses length of sentences and length of words.
    Higher is for more advanced readers.
    If text is sparse i.e. mostly new lines, and doesn't end with an eos -> add a negative cost.  
    Args:
        text (str): original text to return score for.
        texts_sentences (list): text split to sentences. 
    """
    txt_words = split_words(text)
    num_letters = sum(len(word) for word in txt_words)
    num_words = len(txt_words)
    num_sent = len(texts_sentences)

    # check if a "sparse" sentence
    if num_sent == 1:
        new_line_threshold = 0 if num_words == 0 else num_words // 4
        if texts_sentences[0].count('\n') > new_line_threshold or not re.search(r'(?<![A-Z])[.!?;"]+', texts_sentences[0]):
            num_sent = 0

    letters_per_word = -10 if num_words == 0 else num_letters/num_words
    words_per_sentence = -10 if num_sent == 0 else num_words/num_sent
    # 0.5 to weight words_per_sentence higher
    return 0.5*letters_per_word + words_per_sentence

In [24]:
!pip install nltk

import nltk
from nltk.stem import WordNetLemmatizer
# Find synonyms, nltk.download('wordnet')
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('omw-1.4')

from nltk import pos_tag

nltk.download('sentiwordnet')
from nltk.corpus import sentiwordnet as swn


You should consider upgrading via the 'C:\Users\tfede\Documents\GitHub\Master-thesis\venv\Scripts\python.exe -m pip install --upgrade pip' command.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tfede\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\tfede\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     C:\Users\tfede\AppData\Roaming\nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!


In [25]:
def _sentiment_polarity(filtered_words):
    """
    Returns a positive sentiment polarity in range 0 = negative/objective to 100 = positive of the entire text.
    Uses SentiWordnet to compute the positiveness polarity of the words and average that value.
    Based on https://nlpforhackers.io/sentiment-analysis-intro/

    Args:
        filtered_words (set): set of non-stop, non-punctuation words. 
    """
    # If  empty
    if len(filtered_words) < 2:
        return 0

    POS_TAG_TO_WN = {'J': wn.ADJ, 'N': wn.NOUN, 'R': wn.ADV, 'V': wn.VERB}
    lemmatizer = WordNetLemmatizer()

    text_sentiment = []
    tagged_words = pos_tag(filtered_words)

    for word, tag in tagged_words:
        wn_tag = POS_TAG_TO_WN.get(tag[0], None)
        if not wn_tag:
            continue

        lemma = lemmatizer.lemmatize(word, pos=wn_tag)
        if not lemma:
            continue

        synsets = wn.synsets(lemma, pos=wn_tag)
        if not synsets:
            continue

        # Take the most common of the synthsets
        synset = synsets[0]
        pos_sentiment = swn.senti_synset(synset.name()).pos_score(
        ) - swn.senti_synset(synset.name()).neg_score()
        text_sentiment.append(pos_sentiment)
    return 0 if not text_sentiment else max(int(np.mean(text_sentiment)*100), 0)

In [26]:
def _simplicity(filtered_words_set):
    """
    Fraction of most frequent words from generated text.
    Args:
        filtered_words_set (set): set of non-stop, non-punctuation words. 
    """
    SEVEN_PREC_MOST_FREQ_WORDS = {'six', 'month', 'drew', 'want', 'hands', 'staring', 'guests', 'goose', 'fitted', 'rope', 'grace', 'delightful', 'meg', 'peace', 'lovely', 'iron', 'dark', 'cloak', 'pictures', 'eaten', 'sake', 'hurt', 'soldiers', 'dragon', 'late', 'unusual', 'centre', 'shore', 'gloomy', 'burning', 'time', 'foreign', 'bride', 'show', 'disappeared', 'light', 'spirits', 'arose', 'larger', 'sunshine', 'paul', 'cries', 'nearest', 'refuse', 'cut', 'fun', 'naughty', 'ears', 'remember', 'filled', 'playing', 'ask', 'loud', 'suggested', 'husband', 'placed', 'proud', 'places', 'difficult', 'somebody', 'eat', 'fault', 'school', 'honor', 'maybe', 'faith', 'win', 'full', 'missed', 'big', 'pieces', 'asking', 'case', 'unto', 'wolf', 'jennings', 'color', 'riding', 'elder', 'torn', 'stay', 'monkeys', 'comes', 'largest', 'crew', 'receive', 'feast', 'vast', 'foot', 'office', 'cake', 'throughout', 'indians', 'trust', 'cheerful', 'building', 'star', 'apple', 'younger', 'henry', 'matters', 'surface', 'york', 'summer', 'nothing', 'mentioned', 'come', 'hope', 'certain', 'seized', 'folded', 'jack', 'absolutely', 'surrounded', 'noise', 'hardly', 'entrance', 'hold', 'pass', 'subject', 'three', 'coming', 'rolling', 'mouse', 'sense', 'alarmed', 'influence', 'day', 'beneath', 'ceased', 'minchin', 'wood', 'laughter', 'difficulty', 'merry', 'marionette', 'state', 'dangerous', 'fought', 'lesson', 'ones', 'upper', 'house', 'mighty', 'ever', 'fountain', 'path', 'loss', 'help', 'present', 'tore', 'beat', 'princes', 'company', 'knees', 'wilt', 'charles', 'finished', 'blame', 'doors', 'replied', 'wait', 'cruel', 'glass', 'chest', 'extraordinary', 'mad', 'explain', 'eldest', 'sank', 'shed', 'driven', 'fall', 'prove', 'rising', 'angry', 'pulling', 'field', 'grandmother', 'mysterious', 'tea', 'sought', 'natural', 'raised', 'learned', 'must', 'hook', 'cared', 'gained', 'monster', 'beth', 'nearly', 'gazing', 'heap', 'rabbit', 'monsieur', 'stars', 'said', 'soft', 'jolly', 'stiff', 'exceedingly', 'us', 'named', 'smoke', 'colia', 'hurried', 'discover', 'close', 'removed', 'explained', 'crept', 'cutting', 'sight', 'mere', 'leaf', 'world', 'witch', 'putting', 'awful', 'grandfather', 'daughters', 'mountain', 'amid', 'understood', 'whispered', 'anger', 'hole', 'hunting', 'pretended', 'evening', 'shouted', 'sword', 'animal', 'swam', 'instant', 'madame', 'chariot', 'fairy', 'eager', 'master', 'raise', 'hercules', 'splendid', 'good', 'pocket', 'betty', 'bowed', 'thanks', 'regular', 'sorry', 'showed', 'dish', 'actually', 'place', 'sperm', 'water', 'aglaya', 'needs', 'pink', 'tumbled', 'ancient', 'rosy', 'sadly', 'else', 'lighted', 'sort', 'wall', 'dreamed', 'warm', 'bid', 'elinor', 'ice', 'week', 'supper', 'wounded', 'sell', 'refused', 'cook', 'otherwise', 'shoulder', 'glanced', 'dropped', 'stand', 'pretend', 'friends', 'skin', 'keep', 'call', 'beard', 'drawn', 'drawing', 'lying', 'move', 'married', 'send', 'edward', 'possibly', 'horrid', 'think', 'courage', 'great', 'work', 'gently', 'formed', 'society', 'rather', 'amy', 'branches', 'hopes', 'precious', 'depths', 'names', 'anxious', 'saw', 'cow', 'blue', 'forgot', 'talking', 'road', 'lose', 'rajah', 'attack', 'follow', 'third', 'leaves', 'waves', 'hang', 'jo', 'fancied', 'treasure', 'head', 'whatever', 'devil', 'fully', 'buried', 'wet', 'gun', 'looked', 'brought', 'next', 'farther', 'decided', 'band', 'cheeks', 'steal', 'sent', 'tossed', 'hear', 'maiden', 'servant', 'ride', 'tiny', 'feared', 'wave', 'travelled', 'one', 'green', 'teach', 'fro', 'tell', 'cool', 'cunning', 'catch', 'merely', 'badger', 'stepped', 'shining', 'owner', 'observe', 'chanced', 'bottom', 'hero', 'heavy', 'land', 'grateful', 'strange', 'circumstances', 'dwarf', 'quick', 'miserable', 'note', 'ship', 'fallen', 'word', 'john', 'turned', 'inclined', 'rat', 'easy', 'legs', 'bring', 'empty', 'touching', 'drinking', 'bade', 'army', 'sure', 'fierce', 'presented', 'key', 'tender', 'nobody', 'directly', 'build', 'edge', 'hidden', 'experience', 'begun', 'attempt', 'miles', 'violent', 'running', 'firmly', 'kicked', 'lived', 'party', 'chamber', 'savage', 'usually', 'drove', 'concluded', 'arranged', 'baby', 'pick', 'fail', 'england', 'reading', 'wants', 'generally', 'red', 'working', 'creatures', 'flew', 'keen', 'person', 'toad', 'related', 'twelve', 'aunt', 'confidence', 'moved', 'took', 'horrible', 'leaning', 'confess', 'spring', 'standing', 'vessel', 'locked', 'scarecrow', 'drank', 'reckon', 'happen', 'received', 'group', 'lebedeff', 'sun', 'sons', 'welcome', 'youngest', 'solid', 'impossible', 'princess', 'sky', 'learning', 'tongue', 'silly', 'began', 'fell', 'following', 'upstairs', 'moon', 'twice', 'given', 'everywhere', 'together', 'bell', 'heat', 'rough', 'gentle', 'carefully', 'longer', 'months', 'none', 'delicate', 'rest', 'darted', 'say', 'alone', 'former', 'everyone', 'rested', 'paid', 'devoted', 'understand', 'nodded', 'winter', 'animals', 'o', 'camp', 'consider', 'demanded', 'odd', 'pull', 'read', 'sold', 'bite', 'innocent', 'waters', 'heard', 'rode', 'stretched', 'gay', 'peculiar', 'position', 'done', 'shoes', 'express', 'streets', 'believe', 'later', 'fact', 'sea', 'seemed', 'drops', 'heaven', 'sail', 'hunt', 'grant', 'hans', 'grave', 'private', 'pipe', 'clock', 'put', 'taught', 'travel', 'beginning', 'slightly', 'view', 'slowly', 'blew', 'kept', 'plenty', 'bought', 'mind', 'belong', 'slow', 'eggs', 'special', 'happily', 'yellow', 'two', 'due', 'clear', 'handsome', 'deeply', 'short', 'cast', 'belonged', 'bore', 'enter', 'spread', 'shut', 'fill', 'closely', 'excited', 'capable', 'arm', 'obtained', 'seven', 'earth', 'truly', 'silence', 'matter', 'box', 'prisoner', 'gates', 'approaching', 'situation', 'mamma', 'gathered', 'mile', 'lot', 'way', 'clouds', 'song', 'without', 'distant', 'shoulders', 'kiss', 'cottage', 'armed', 'within', 'trying', 'smile', 'got', 'gate', 'eagle', 'neither', 'stands', 'somewhat', 'walking', 'asked', 'stubb', 'agreed', 'forgive', 'charge', 'bad', 'sprang', 'quite', 'washed', 'tear', 'retired', 'river', 'except', 'scarcely', 'landed', 'object', 'attracted', 'cock', 'rid', 'row', 'weight', 'watch', 'many', 'solitary', 'table', 'deeper', 'whether', 'swear', 'nearer', 'touched', 'colin', 'pleased', 'dreadful', 'false', 'ozma', 'dig', 'dorothy', 'future', 'cost', 'possible', 'anybody', 'offered', 'thy', 'sitting', 'line', 'darkness', 'kissed', 'summoned', 'laying', 'led', 'spell', 'considered', 'uncle', 'accustomed', 'marriage', 'whenever', 'pay', 'died', 'treated', 'hate', 'join', 'amuse', 'cried', 'finest', 'walk', 'things', 'won', 'hastened', 'leaped', 'emerald', 'swimming', 'affection', 'pounds', 'guard', 'swung', 'desert', 'bright', 'cure', 'laughing', 'piece', 'faster', 'hair', 'older', 'indeed', 'remain', 'reason', 'various', 'stuff', 'waving', 'comfortable', 'rise', 'settled', 'fool', 'friend', 'earnestly', 'island', 'comfort', 'beloved', 'kind', 'stayed', 'tin', 'happiness', 'ground', 'liked', 'sometimes', 'ivan', 'personal', 'longed', 'thing', 'meeting', 'simple', 'always', 'hunter', 'roof', 'obey', 'hot', 'delighted', 'moment', 'struck', 'cause', 'chance', 'whole', 'discovered', 'roll', 'seat', 'kitchen', 'bottle', 'taking', 'dickon', 'important', 'young', 'forget', 'bearing', 'safe', 'soon', 'deep', 'turning', 'holy', 'pinocchio', 'repeated', 'lady', 'papa', 'ways', 'crown', 'right', 'thou', 'body', 'dog', 'somewhere', 'steps', 'clearly', 'wedding', 'grey', 'cloth', 'mine', 'character', 'kindly', 'fish', 'silk', 'amongst', 'apparently', 'thinks', 'horse', 'latter', 'dim', 'wear', 'found', 'hurry', 'distance', 'perceived', 'apples', 'appeared', 'creature', 'form', 'tired', 'acquaintance', 'pale', 'remained', 'corner', 'moving', 'five', 'bless', 'queen', 'reward', 'defarge', 'kingdom', 'hit', 'governor', 'rubbed', 'stream', 'danced', 'host', 'bull', 'pressed', 'picture', 'escaped', 'crow', 'hall', 'wishing', 'persuaded', 'turkey', 'mortal', 'difference', 'laughed', 'thank', 'hand', 'front', 'dost', 'study', 'fly', 'nest', 'inquired', 'hours', 'burned', 'fetch', 'city', 'just', 'mischief', 'gradually', 'hated', 'claim', 'telling', 'yonder', 'faint', 'papers', 'meet', 'quickly', 'smith', 'plain', 'thrust', 'ran', 'entire', 'unhappy', 'lay', 'boat', 'roared', 'forty', 'wise', 'excellent', 'listened', 'vanished', 'climbed', 'manner', 'unless', 'parents', 'gift', 'rocks', 'force', 'low', 'garden', 'door', 'much', 'gray', 'greatest', 'wretched', 'year', 'angel', 'returned', 'stuck', 'waited', 'little', 'held', 'interested', 'fiery', 'purpose', 'gazed', 'woke', 'especially', 'recovered', 'useful', 'terribly', 'sad', 'wherever', 'might', 'carriage', 'maid', 'around', 'left', 'partly', 'sensible', 'ashamed', 'ordered', 'hollow', 'last', 'value', 'may', 'notice', 'troubles', 'mountains', 'wondered', 'weather', 'night', 'remarkable', 'slip', 'grow', 'floor', 'man', 'divided', 'wizard', 'highly', 'marched', 'rose', 'climb', 'fox', 'lucy', 'becoming', 'rushing', 'however', 'acted', 'number', 'meaning', 'orders', 'means', 'step', 'mention', 'tie', 'flowers', 'forced', 'promise', 'hoped', 'let', 'lie', 'poor', 'god', 'bundle', 'sacred', 'meantime', 'either', 'evil', 'white', 'game', 'return', 'invited', 'examined', 'swallowed', 'brave', 'shone', 'doctor', 'pointed', 'leg', 'deal', 'excuse', 'greater', 'yard', 'knock', 'accepted', 'share', 'wearing', 'proved', 'leading', 'still', 'books', 'seek', 'dying', 'higher', 'lion', 'flock', 'cross', 'favorite', 'laugh', 'see', 'houses', 'painful', 'stronger', 'adventure', 'parts', 'sat', 'bread', 'appear', 'sudden', 'proper', 'reach', 'worked', 'turns', 'memory', 'among', 'appearance', 'men', 'give', 'listen', 'heels', 'keeping', 'tsar', 'better', 'lead', 'point', 'best', 'guess', 'live', 'lift', 'leaving', 'write', 'shape', 'lifted', 'intention', 'blowing', 'stories', 'woman', 'windows', 'needed', 'bird', 'thither', 'cover', 'tears', 'allowed', 'wound', 'singing', 'lower', 'suppose', 'rain', 'room', 'hour', 'trouble', 'threw', 'tells', 'tall', 'top', 'sweet', 'fighting', 'kill', 'already', 'jason', 'tree', 'united', 'behind', 'shown', 'almost', 'danger', 'native', 'delicious', 'basket', 'dermat', 'leave', 'utter', 'pleasant', 'everything', 'wrote', 'knight', 'becky', 'vain', 'believed', 'bill', 'peter', 'music', 'ka', 'address', 'prepared', 'stupid', 'perfectly', 'admit', 'played',
                              'beside', 'finally', 'wept', 'familiar', 'suffered', 'everybody', 'similar', 'though', 'fled', 'passing', 'law', 'imagine', 'succeeded', 'alive', 'usual', 'gives', 'battle', 'thoroughly', 'rooms', 'gone', 'followed', 'carrying', 'effort', 'rich', 'bent', 'hastily', 'prison', 'square', 'hid', 'along', 'eight', 'eagerly', 'fond', 'since', 'dared', 'trembling', 'bare', 'smooth', 'away', 'loaded', 'south', 'thought', 'boots', 'black', 'busy', 'stone', 'van', 'presence', 'sounded', 'bound', 'pleasure', 'idea', 'wooden', 'death', 'ordinary', 'willing', 'original', 'arms', 'added', 'spare', 'losing', 'told', 'money', 'fair', 'quarter', 'words', 'hannah', 'voice', 'knife', 'getting', 'peeped', 'free', 'sees', 'street', 'dug', 'monkey', 'frightened', 'grown', 'dogs', 'weary', 'fellow', 'pulled', 'finger', 'dwelt', 'resolved', 'woods', 'pure', 'according', 'tail', 'loving', 'drive', 'habit', 'magician', 'thinking', 'admitted', 'jumped', 'joy', 'play', 'kinds', 'curiosity', 'returning', 'towards', 'size', 'lucky', 'major', 'sailing', 'interest', 'name', 'days', 'declared', 'spoken', 'plan', 'palace', 'castle', 'trembled', 'court', 'chair', 'equal', 'ahab', 'known', 'announced', 'passage', 'old', 'run', 'hoping', 'rush', 'lofty', 'talk', 'powerful', 'truth', 'george', 'attend', 'fat', 'fate', 'clothes', 'news', 'thief', 'taken', 'new', 'queer', 'circle', 'space', 'strength', 'lives', 'hanging', 'something', 'need', 'feet', 'awoke', 'whoever', 'waiting', 'sir', 'rage', 'temple', 'air', 'bench', 'early', 'stick', 'fear', 'sorrow', 'feel', 'soul', 'stout', 'shoot', 'nurse', 'convinced', 'thanksgiving', 'daughter', 'king', 'service', 'thousand', 'opening', 'shaking', 'flower', 'opportunity', 'minutes', 'ago', 'real', 'probably', 'gets', 'near', 'simply', 'ended', 'smiling', 'someone', 'sister', 'sum', 'whilst', 'enough', 'crying', 'writing', 'heart', 'silent', 'addressed', 'marianne', 'occasion', 'pity', 'forehead', 'west', 'u', 'somehow', 'marble', 'mark', 'faithful', 'mole', 'dream', 'bow', 'slight', 'brass', 'find', 'anything', 'boys', 'fingers', 'weak', 'wanted', 'caused', 'across', 'fed', 'side', 'outside', 'persons', 'grief', 'bushes', 'excitement', 'softly', 'charming', 'paused', 'eyes', 'foolish', 'oak', 'even', 'interrupted', 'finding', 'forest', 'buy', 'johnny', 'ten', 'lips', 'back', 'smiled', 'nights', 'muttered', 'walls', 'fifteen', 'hast', 'shame', 'thick', 'enemy', 'gave', 'thrown', 'task', 'knowledge', 'cold', 'certainly', 'like', 'shadows', 'enjoy', 'wine', 'pig', 'far', 'darling', 'fellows', 'pursued', 'snake', 'wendy', 'turn', 'looking', 'worthy', 'slightest', 'toto', 'striking', 'slept', 'throne', 'forward', 'hearts', 'slipped', 'figure', 'shelter', 'direction', 'attention', 'question', 'handed', 'treat', 'beg', 'runs', 'candle', 'fourth', 'interesting', 'bag', 'huge', 'hut', 'worse', 'knocked', 'years', 'magnificent', 'different', 'spend', 'whose', 'sound', 'miss', 'lit', 'small', 'lad', 'burn', 'wish', 'general', 'necessary', 'fields', 'clever', 'beasts', 'serious', 'sleeping', 'fixed', 'please', 'use', 'murmured', 'saint', 'went', 'throw', 'commanded', 'four', 'fresh', 'ball', 'giving', 'flung', 'arrived', 'sighed', 'carried', 'french', 'arrival', 'afraid', 'burnt', 'hung', 'exactly', 'cloud', 'face', 'calm', 'perfect', 'mean', 'er', 'robbers', 'strangely', 'rushed', 'hard', 'another', 'merchant', 'girls', 'continually', 'lizabetha', 'sack', 'milk', 'lord', 'common', 'window', 'fur', 'round', 'honour', 'ass', 'fight', 'human', 'recognized', 'nine', 'makes', 'dressed', 'suddenly', 'waved', 'double', 'avoid', 'seems', 'evgenie', 'morning', 'string', 'impression', 'conversation', 'glancing', 'middle', 'eleven', 'brother', 'seen', 'neck', 'search', 'weeks', 'started', 'stood', 'sides', 'dare', 'painted', 'miller', 'art', 'knowing', 'wash', 'midnight', 'became', 'fire', 'agreeable', 'soldier', 'blow', 'family', 'minute', 'coat', 'stop', 'beast', 'joe', 'true', 'others', 'robin', 'able', 'walked', 'shook', 'part', 'hat', 'wishes', 'peasant', 'people', 'ugly', 'jungle', 'surprise', 'evidently', 'board', 'closed', 'sing', 'loved', 'sang', 'rolled', 'crowd', 'serve', 'moments', 'immense', 'happy', 'anyone', 'stole', 'therefore', 'sheep', 'often', 'bank', 'hundred', 'meat', 'possessed', 'chase', 'remembered', 'pack', 'horror', 'stranger', 'heavily', 'wicked', 'mother', 'voyage', 'glance', 'dozen', 'visit', 'opinion', 'constantly', 'deck', 'single', 'holding', 'whales', 'enchanted', 'care', 'gods', 'considerable', 'food', 'beating', 'silver', 'colonel', 'feed', 'loves', 'become', 'faces', 'events', 'worst', 'using', 'rare', 'pine', 'grew', 'dinner', 'mud', 'driving', 'oil', 'purple', 'suffer', 'letter', 'narrow', 'sounds', 'drop', 'wished', 'sick', 'blood', 'mice', 'tiger', 'companion', 'every', 'noble', 'escape', 'lake', 'make', 'approached', 'aside', 'remarked', 'secret', 'sooner', 'ready', 'felt', 'tower', 'longing', 'spoke', 'bear', 'fifty', 'growing', 'visited', 'long', 'christmas', 'several', 'past', 'stones', 'greatly', 'born', 'frog', 'obliged', 'ere', 'joined', 'business', 'fine', 'dust', 'folks', 'beautiful', 'worn', 'holmes', 'wake', 'touch', 'twenty', 'glad', 'grand', 'enjoyed', 'satisfied', 'finds', 'valuable', 'less', 'uttered', 'straight', 'respect', 'mowgli', 'points', 'wind', 'thus', 'life', 'can', 'golden', 'branch', 'ships', 'afternoon', 'wondering', 'know', 'steady', 'change', 'hare', 'fancy', 'rapidly', 'proceeded', 'nose', 'helped', 'killed', 'expected', 'train', 'delight', 'de', 'feathers', 'tom', 'flying', 'midst', 'showing', 'observed', 'changed', 'chosen', 'dear', 'enormous', 'setting', 'lines', 'likewise', 'lamp', 'seldom', 'condition', 'listening', 'saying', 'equally', 'speak', 'cup', 'quietly', 'failed', 'came', 'surprised', 'captain', 'comrade', 'fairly', 'prevent', 'expression', 'continued', 'reached', 'gold', 'winged', 'thee', 'entirely', 'youth', 'waste', 'laurie', 'companions', 'picked', 'thin', 'pushed', 'finish', 'rock', 'stopped', 'seated', 'occurred', 'nastasia', 'drink', 'children', 'hind', 'broke', 'voices', 'covered', 'history', 'seeing', 'seem', 'paper', 'wife', 'half', 'breath', 'fairies', 'eating', 'flat', 'fastened', 'increased', 'exclaimed', 'possession', 'mary', 'hungry', 'assured', 'frequently', 'watched', 'lights', 'hearty', 'anywhere', 'sleep', 'bones', 'daily', 'son', 'trunk', 'promised', 'answer', 'marry', 'fit', 'first', 'leaned', 'straw', 'safely', 'luck', 'making', 'nice', 'handkerchief', 'take', 'epanchin', 'silently', 'breaking', 'thirty', 'open', 'huntsman', 'boy', 'swift', 'stared', 'nature', 'lorry', 'poured', 'tied', 'intended', 'delivered', 'wide', 'town', 'public', 'rang', 'mounted', 'horses', 'lies', 'concealed', 'made', 'disturbed', 'chief', 'speaking', 'north', 'spent', 'instantly', 'girl', 'goes', 'snow', 'main', 'expressed', 'tried', 'birds', 'completely', 'loudly', 'loose', 'bed', 'ate', 'called', 'blessed', 'begged', 'home', 'couple', 'shadow', 'bigger', 'queequeg', 'pitcher', 'bits', 'content', 'says', 'desire', 'sisters', 'mount', 'falling', 'dress', 'glimpse', 'march', 'anxiety', 'meal', 'command', 'kindness', 'dirty', 'war', 'beauty', 'sailed', 'worth', 'careful', 'accept', 'allow', 'complete', 'idle', 'travelling', 'ye', 'going', 'assure', 'suffering', 'spirit', 'woodman', 'particularly', 'harm', 'village', 'ear', 'pretty', 'image', 'wandering', 'honest', 'letters', 'stopping', 'cave', 'seeking', 'dead', 'set', 'tale', 'hunger', 'high', 'awake', 'sending', 'london', 'brothers', 'broad', 'dancing', 'farmer', 'smell', 'peaceful', 'breakfast', 'calls', 'also', 'gentleman', 'questions', 'inside', 'stolen', 'count', 'famous', 'regard', 'broken', 'speech', 'end', 'plunged', 'unable', 'gania', 'wonderful', 'shake', 'second', 'large', 'now', 'gladly', 'pair', 'clean', 'mistress', 'gather', 'watching', 'rogojin', 'pan', 'afterwards', 'deadly', 'supposed', 'signs', 'eye', 'funny', 'unfortunate', 'choose', 'troubled', 'bold', 'altogether', 'used', 'shall', 'trees', 'sorts', 'flight', 'twisted', 'crossed', 'guessed', 'forth', 'fast', 'hearing', 'owl', 'brown', 'pointing', 'well', 'fetched', 'throwing', 'love', 'screamed', 'haste', 'sara', 'least', 'opened', 'race', 'takes', 'dashwood', 'sharp', 'prince', 'taste', 'served', 'will', 'satisfaction', 'beyond', 'giant', 'wandered', 'shot', 'sign', 'household', 'capital', 'perhaps', 'meant', 'corn', 'duty', 'advice', 'informed', 'managed', 'regarded', 'spot', 'servants', 'book', 'ladies', 'dashed', 'mouth', 'yesterday', 'scene', 'floating', 'axe', 'offer', 'swiftly', 'feelings', 'power', 'hill', 'look', 'age', 'gentlemen', 'thoughts', 'lively', 'never', 'stairs', 'go', 'roses', 'fortune', 'knows', 'doubt', 'living', 'scattered', 'reply', 'draw', 'immediately', 'considering', 'engaged', 'sunday', 'brilliant', 'cat', 'learn', 'suspected', 'met', 'quiet', 'women', 'dry', 'majesty', 'forgotten', 'tailor', 'looks', 'saved', 'startled', 'pray', 'father', 'friendly', 'conduct', 'astonished', 'lump', 'passed', 'begin', 'manners', 'caught', 'child', 'upon', 'plainly', 'widow', 'provided', 'boats', 'starting', 'pot', 'manage', 'likely', 'besides', 'dance', 'solemn', 'creeping', 'curious', 'opposite', 'account', 'particular', 'written', 'burst', 'ring', 'required', 'till', 'start', 'ill', 'journey', 'settle', 'answered', 'unknown', 'yards', 'learnt', 'asleep', 'jump', 'wonder', 'strong', 'dollars', 'beheld', 'breast', 'story', 'terror', 'dull', 'expect', 'easily', 'presently', 'marked', 'swept', 'try', 'judge', 'bringing', 'carry', 'hills', 'lonely', 'american', 'happened', 'sit', 'country', 'length', 'wrong', 'wings', 'valley', 'aware', 'times', 'spite', 'lest', 'instead', 'oh', 'storm', 'feeling', 'royal', 'arrow', 'terrible', 'naturally', 'wore', 'intend', 'hide', 'whence', 'east', 'cap', 'yet', 'toward', 'floated', 'noticed', 'course', 'whale', 'pain', 'pride', 'canst', 'desired', 'surely', 'dragged', 'aid', 'magic', 'determined', 'lost', 'dried', 'entered', 'save', 'grass', 'oz', 'stir', 'break', 'really', 'blind', 'act', 'built', 'keeps', 'produced', 'although', 'tone', 'suit', 'order', 'emperor', 'lodge', 'bit', 'calling', 'bitter', 'cry', 'sympathy', 'extremely', 'kid', 'laid', 'heads', 'giants', 'wrapped', 'teeth', 'get', 'wild', 'talked', 'die', 'effect', 'church', 'strike', 'fruit', 'thanked', 'knew', 'respectable'}

    return len(filtered_words_set.intersection(SEVEN_PREC_MOST_FREQ_WORDS))

In [27]:
def _diversity(filtered_words, filtered_words_set):
    """
    Fraction of unique words from the total number of words (exclusing stop words).
    Higher is more diversified.
    Args:
        filtered_words (list): set of non-stop tokenized words. 
        filtered_words_set (set): unique filtered words.
    """
    MIN_WORDS_PER_STORY = 5
    # If empty sentence or only white space or \n or too repetitive.
    if len(filtered_words_set) < MIN_WORDS_PER_STORY:
        return 0

    return len(filtered_words_set) / len(filtered_words)

In [28]:
from torch.nn import KLDivLoss, Softmax, LogSoftmax

def _load_KLDIV_loss_function(device):
    """
    Load loss function and its utilities.
    """
    loss_fct = torch.nn.KLDivLoss(reduction='batchmean')
    softmax = Softmax(dim=-1)
    logSoftmax = LogSoftmax(dim=-1)
    loss_fct.to(device)
    softmax.to(device)
    logSoftmax.to(device)
    return loss_fct, softmax, logSoftmax

In [29]:
def KLDIV_error_per_text(tokenizer, preset_model, finetuned_model, text):
    """
    Computes the difference in prediction scores of the given text between the two models.
    The preset_model scores are the input distribution and the finetuned_model scores are the target distribution.
    The forward pass of each model returns the "prediction scores of the language modeling head (scores for each vocabulary token before SoftMax)" (from: https://huggingface.co/transformers/model_doc/gpt2.html).

    Args:
        tokenizer (Pytroch tokenizer): GPT2 Byte Tokenizer. 
        preset_model (Pytorch model): preset GPT2 model of the same/ different size of the finetuned model. Assumes model max_length < num_of_words(text).
        finetuned_model (Pytorch model): fine-tuned GPT2 model. Assumes model max_length < num_of_words(text).
        text (str): generated text to check predictions scores for.
    Returns:
        float representing the difference in scores. Bigger is probably better since it usually means the text is closer to the finetuned distribution.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    # Text is divided to extracts e.g. start, middle, end.
    if isinstance(text, (list, np.ndarray)):
        text = ' '.join(text)

    # if too short return 0
    if len(text) < 10:
        return 0

    # Prepare text to forward pass.
    encodings_dict = tokenizer(text)
    text_ids = torch.tensor(
        encodings_dict['input_ids'], device=device, dtype=torch.long)

    # Load loss function, the losses are averaged over batch dimension.
    loss_fct, softmax, logSoftmax = _load_KLDIV_loss_function(device)
    # unsqueeze to add batch_size =1
    # zero index to logits, logits shape (batch_size, len(text_ids), config.vocab_size)
    logists_preset = preset_model(text_ids.to(device).unsqueeze(0))[0]
    logits_finetuned = finetuned_model(text_ids.to(device).unsqueeze(0))[0]

    # input should be in log-probabilities, and target in probabilities.
    return loss_fct(logSoftmax(logists_preset), softmax(logits_finetuned)).item()

In [30]:
!pip install -U scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer


You should consider upgrading via the 'C:\Users\tfede\Documents\GitHub\Master-thesis\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [62]:
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity

def _coherency(texts_sentences):
    """
    Args:
        texts_sentences (list): one story text split to sentences.
        lsa (sklearn Vectorizer).
    Based on LSA (TF-IDF -> Truncated SVD), returns the similarity within the story setences in comparison to the first sentence.
    """
    vectorizer = TfidfVectorizer()
    # Compute tf-idf per extract.
#     transformed_sentences = embedder(texts_sentences)
    transformed_sentences = vectorizer.fit_transform(texts_sentences)
    lsa_model = TruncatedSVD(n_components=3, algorithm='arpack', n_iter=10, random_state=42)

    lsa_top=lsa_model.fit_transform(transformed_sentences)

    # Compute cosine max similarity per extract, shape (#texts_sentences x #texts_sentences).
    similarity = cosine_similarity(lsa_top)
    # Compute similarity scores with first sentence of the rest of the sentences.
    return sum(similarity[0][1:])

In [32]:
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M").cuda()
device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")
sentence = ["I want to be a millionaire "]
first_idx = False
encodings_dict = tokenizer(sentence)
sliced_inputs = [encodings_dict['input_ids'][0][-550:]]

prompts_ids = torch.tensor(
    sliced_inputs, device=device, dtype=torch.long)
first_idx = len(prompts_ids[0]) if first_idx else 0


In [33]:
def _preprocess_generated_text(sample, tokenizer, has_space):
    decoded = tokenizer.decode(
        sample, skip_special_tokens=True)
    # Removing spaces.
    decoded = decoded.strip()
    # Adding a space at the beginning if needed.
    if not has_space:
        decoded = ' ' + decoded
    # Filtering � globally
    return re.sub(u'\uFFFD', '', decoded)

In [34]:
max_length = 25
num_return_sequences=10
sample_outputs = model.generate(
    prompts_ids,  # Long tensor of size (batch_size, max_prompt_length)
    do_sample=True,  # activate top-k, top-p sampling
    max_length=max_length+first_idx,
    min_length=first_idx + max_length//2 if first_idx else 10,
    top_k=70,
    top_p=0.95,
    temperature=1.05,
    repetition_penalty=1.0,  # no penalty
    num_return_sequences=num_return_sequences,
    pad_token_id=tokenizer.pad_token_id,
)  # returns tensor of shape (len(prompts)*num_return_sequences x max_length)
has_space = sentence[0][-1].isspace()
generated = map(lambda sample: _preprocess_generated_text(
    sample[first_idx:], tokenizer, has_space), sample_outputs)
generated = np.array(
    list(filter(lambda sample: len(sample.strip()) > 2, generated)))
print(generated)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['I want to be a millionaire  \nnow       \nand I will get rich'
 'I want to be a millionaire          \n  1. Do not have the strength'
 'I want to be a millionaire!" "What was that?" "What had he done to that man?" "What,'
 "I want to be a millionaire \nno one thinks its enough of you\nyou're the best you are"
 "I want to be a millionaire \nwith a big name.\nBecause you don't get to choose who\nyour money"
 'I want to be a millionaire  \n     and I want to do the right things'
 'I want to be a millionaire?"\n\nHe was looking up. He felt his teeth breaking out. He was in'
 'I want to be a millionaire!!! So you come in and tell me if you really want to become a millionaire or'
 'I want to be a millionaire  \nfor the money I spend.  \nI want to know whether I can'
 'I want to be a millionaire']


In [35]:
!pip install stop-words

from stop_words import get_stop_words
STOP_WORDS = set(get_stop_words('english'))

You should consider upgrading via the 'C:\Users\tfede\Documents\GitHub\Master-thesis\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [36]:
def score_text(text, tokenizer, preset_model, finetuned_model):
# def score_text(text):
    """ Uses rule-based rankings. Higher is better, but different features have different scales.

    Args:
        text (str/ List[str]): one story to rank.
        tokenizer (Pytroch tokenizer): GPT2 Byte Tokenizer. 
        preset_model (Pytorch model): preset GPT2 model of the same/ different size of the finetuned model. 
        finetuned_model (Pytorch model): fine-tuned GPT2 model. 

    Returns a scores np.array of corresponding to text.
    """
    assert isinstance(
        text, (str, list)), f"score_text accepts type(text) = str/list, but got {type(text)}"

    if isinstance(text, list):
        text = ' '.join(text)

    # Keep same order as in constants.FEATURES
    scores = [0 for _ in range(6)]
    texts_sentences = split_to_sentences(text)
    scores[0] = _coherency(texts_sentences)
    scores[1] = _readabilty(text, texts_sentences)

#     Set of text words without punctuation and stop words.
    filtered_words = list(filter(
        lambda word: word not in STOP_WORDS, split_words(text.lower().strip())))
    filtered_words_set = set(filtered_words)
    # Sentiment.
    scores[2] = _sentiment_polarity(filtered_words)

    # Set based measures.
    scores[3] = _simplicity(filtered_words_set)
    scores[4] = _diversity(filtered_words, filtered_words_set)

#     # The bigger differene, the more tale-like, similar to the fine-tuned model, the text is.
    scores[5] = KLDIV_error_per_text(
        tokenizer, preset_model, finetuned_model, text)

#     # print(" | ".join(f'{key}: {score:.2f}' for key,
#     #                  score in zip(constants.FEATURES, scores)))

    return np.array(scores)

In [37]:
def sort_scores(stories_scores):
    """
    Args:
        stories_scores (np.array): 2D matrix of shpae (#stories x #ranking_features).
    Returns the indices of top stories accroding to scores, from highest to lowest (descending).
    """
    # Rescale each feature column across all stories, so that all featues contribute equally.
    # stories_scores_std = (stories_scores - np.mean(stories_scores,axis=0))/np.std(stories_scores,axis=0)
    stories_scores_normalized = stories_scores - \
        np.min(stories_scores, axis=0)
    min_max_denominator = np.max(
        stories_scores, axis=0) - np.min(stories_scores, axis=0)
    # Avoid devision by zero, out to initialize idices where denominator ==0.
    stories_scores_normalized = np.divide(
        stories_scores_normalized, min_max_denominator, out=np.zeros_like(stories_scores_normalized), where=min_max_denominator != 0)

    # Sort by mean story score, shape: (num_stories)
    return np.argsort(np.mean(stories_scores_normalized, axis=1))[::-1]

In [60]:
stories_scores = np.array(list(map(lambda text: score_text(
    text, tokenizer, model, model), generated)))
stories_scores


c:\users\tfede\documents\github\master-thesis\venv\lib\site-packages\sklearn\decomposition\_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
c:\users\tfede\documents\github\master-thesis\venv\lib\site-packages\sklearn\decomposition\_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
c:\users\tfede\documents\github\master-thesis\venv\lib\site-packages\sklearn\decomposition\_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var
c:\users\tfede\documents\github\master-thesis\venv\lib\site-packages\sklearn\decomposition\_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


array([[ 0.00000000e+00, -7.50000000e+00,  6.00000000e+00,
         4.00000000e+00,  1.00000000e+00, -3.94180617e-08],
       [ 7.69787433e-17,  8.29166667e+00,  3.70000000e+01,
         2.00000000e+00,  0.00000000e+00,  3.40131777e-07],
       [ 1.48500513e-01,  4.53571429e+00,  1.20000000e+01,
         2.00000000e+00,  1.00000000e+00, -1.83561326e-08],
       [ 0.00000000e+00, -7.87500000e+00,  2.90000000e+01,
         5.00000000e+00,  1.00000000e+00,  2.46747646e-07],
       [ 6.80496309e-02,  1.03750000e+01,  0.00000000e+00,
         5.00000000e+00,  1.00000000e+00,  1.44532805e-07],
       [ 0.00000000e+00, -8.07142857e+00,  8.00000000e+00,
         3.00000000e+00,  0.00000000e+00, -1.03521003e-07],
       [-8.47074116e-16,  6.52777778e+00,  0.00000000e+00,
         5.00000000e+00,  1.00000000e+00,  3.44307409e-07],
       [ 2.42919305e-01,  1.28409091e+01,  1.70000000e+01,
         5.00000000e+00,  8.75000000e-01,  3.00840270e-07],
       [ 1.90874066e-01,  1.00937500e+01,  0.000

In [61]:
sorted_idx = sort_scores(stories_scores)
sorted_idx

array([7, 4, 6, 3, 8, 1, 2, 0, 5, 9], dtype=int64)

In [40]:
list(generated[sorted_idx])[:num_return_sequences]

['I want to be a millionaire!!! So you come in and tell me if you really want to become a millionaire or',
 "I want to be a millionaire \nwith a big name.\nBecause you don't get to choose who\nyour money",
 'I want to be a millionaire?"\n\nHe was looking up. He felt his teeth breaking out. He was in',
 "I want to be a millionaire \nno one thinks its enough of you\nyou're the best you are",
 'I want to be a millionaire  \nfor the money I spend.  \nI want to know whether I can',
 'I want to be a millionaire          \n  1. Do not have the strength',
 'I want to be a millionaire!" "What was that?" "What had he done to that man?" "What,',
 'I want to be a millionaire  \nnow       \nand I will get rich',
 'I want to be a millionaire  \n     and I want to do the right things',
 'I want to be a millionaire']